In [ ]:
import socceraction as sa
import soccerdata as sd
from scraper.whoscored_chromeless import WhoScored
from pathlib import PosixPath

In [ ]:
# Setup a scraper for the 2021/2022 Premier League season
ws = WhoScored(
    leagues=["GER-Bundesliga"], # "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", 
    seasons=[21], # 15, 16, 17, 18, 19, 20, 21, 22, 23
    #no_cache=False,
    #no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    # path_to_browser="/usr/bin/chromium",
    #headless=True,
)
api = ws.read_events(output_fmt='spadl')

In [ ]:
api

In [ ]:
games = ws.read_schedule()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from socceraction import spadl, xthreat
events = []
for g, h in zip(games.game_id, games.home_team_id):
    e = api[api["game_id"] == g]
    e = spadl.add_names(e)
    e = spadl.play_left_to_right(e, h)
    events.append(e)


In [ ]:
events_df = pd.concat(events)
events_df

In [ ]:
# Step 3: Train xT model
filtered = events_df[events_df["type_name"].isin(["pass", "carry", "dribble", "shot"])]

In [ ]:
filtered

In [ ]:
filtered = filtered[filtered["end_x"].notna() & filtered["end_y"].notna()]

In [ ]:
filtered

In [ ]:
from socceraction import spadl, xthreat

xTModel = xthreat.ExpectedThreat(l=16, w=12)
xT = xTModel.fit(filtered)

In [ ]:
import seaborn as sns
sns.heatmap(xT.scoring_prob_matrix)

In [ ]:
sns.heatmap(xT.shot_prob_matrix)

In [ ]:
filtered

In [ ]:
xT.rate(filtered)